In [ ]:
import datetime
import pandas as pd

# import log
def import_log_ardupilot(path):
    df = pd.read_csv(path)[['Date', 'Time', 'GSpd(kts)', 'Alt(m)', 'Thr', 'CURR(A)', 'SG']]
    df['time'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df['alt'] = df['Alt(m)'].round(0).astype(int)
    # ASpd & GSpd - These sensors value actually in m/s, not knots, see
    # https://github.com/Clooney82/MavLink_FrSkySPort/wiki/1.2.-FrSky-Taranis-Telemetry
    df['spd'] = (3.6*df['GSpd(kts)']).round(0).astype(int) # m/s to km/h
    df['curr'] = df['CURR(A)']
    df['thr'] = (df['Thr'] / 20.48 + 50).round(0).astype(int) # -1024/1024 to percent
    df.drop(['Date', 'Time', 'GSpd(kts)', 'Alt(m)', 'Thr', 'CURR(A)'], axis=1, inplace=True)
    return df

def import_log_betaflight(path):
    df = pd.read_csv(path)[['Date', 'Time', 'Alt(m)', 'GSpd(kts)', '0420', 'Tmp2(@C)', 'VFAS(V)', 'RSSI(dB)', 'Curr(A)', 'Thr']]
    df['time'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df['alt'] = df['Alt(m)'].round(0).astype(int)
    df['spd'] = (1.852*df['GSpd(kts)']).round(0).astype(int) # knots to km/h
    df['bat'] = df['VFAS(V)'].round(1)
    df['curr'] = df['Curr(A)'].round(0).astype(int)
    df['sats'] = df['Tmp2(@C)'] % 100
    df['thr'] = (df['Thr'] / 20.48 + 50).round(0).astype(int) # -1024/1024 to percent
    df.drop(['GSpd(kts)', 'Alt(m)', 'Thr', 'Curr(A)'], axis=1, inplace=True)
    return df

# create srt
def format_time_srt(s):
    hours, remainder = divmod(s, 3600)
    minutes, remainder = divmod(remainder, 60)
    seconds, remainder = divmod(remainder, 1)
    return '{:02}:{:02}:{:02},{:03}'.format(int(hours), int(minutes), int(seconds), int(remainder*1000))

def format_time_ssa(s):
    hours, remainder = divmod(s, 3600)
    minutes, remainder = divmod(remainder, 60)
    seconds, remainder = divmod(remainder, 1)
    return '{:02}:{:02}:{:02}.{:02}'.format(int(hours), int(minutes), int(seconds), int(remainder*100))

def format_srt(x):
#     + 'SG=' + str(x.SG) + '   '\
    return str(x.name + 1) + '\n' + format_time_srt(x.start) + ' --> ' + format_time_srt(x.end) + '\n'\
    + str(x.alt) + ' m   '\
    + str(x.spd) + ' km/h   '\
    + str(x.curr) + ' A\n\n'

def format_ssa(x):
    s = 'Dialogue: ' + format_time_ssa(x.start) + ',' + format_time_ssa(x.end) + ',A1,{\pos(16,16)}'\
    + '📡 ' + str(x['sats']) + '\\N'\
    + ' ' + str(x['RSSI(dB)']) + '\\N'\
    + ' ' + str(x['0420']) + '\\N'\
    + '\n'
    
    s += 'Dialogue: ' + format_time_ssa(x.start) + ',' + format_time_ssa(x.end) + ',A2,{\pos(16,350)}'\
    + '\\N'\
    + '\\N'\
    + str(x.alt) + ' m\\N'\
    + str(x.spd) + ' ㎞/h\\N'\
    + '\n'
    
    s += 'Dialogue: ' + format_time_ssa(x.start) + ',' + format_time_ssa(x.end) + ',A3,{\pos(16,700)}'\
    + f'🔋{x.curr:2d} A\\N'\
    + f'{x.bat} V\\N'\
    + '\n'
    return s
        
ssa_header = '''[Script Info]
PlayResX: 1280
PlayResY: 720
WrapStyle: 1

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, Alignment, Outline
Style: Default, Segoe UI Symbol,36,&HFFFFFF,5,1
Style: A1, Segoe UI Symbol,36,&HFFFFFF,7,1
Style: A2, Segoe UI Symbol,36,&HFFFFFF,4,1
Style: A3, Segoe UI Symbol,36,&HFFFFFF,1,1

[Events]
Format: Start, End, Style, Text
'''
# Dialogue: 0:00:00.00,0:00:10.00,A2,{\pos(150,400)}10.\N U+E10F\N⌂ U+2302.\N 2:15\N🔋 3.8V\N 4.2V\N 3.6V\N 50
# Battery
# 
    
def export_subtitles_file(df, out_file, formatter):
    ext = '.srt' if formatter == format_srt else '.ssa'
    with open(out_file + ext, "w", encoding='utf-8') as f:
        if formatter == format_ssa:
            f.write(ssa_header)
        for index, row in df.iterrows():
            f.write(formatter(row))

def export_subtitles(df, out_file, formatter=format_ssa, skip_log_rows=0, speed_correction=1, shift_sec=0, split_sec=0):
    max_length_sec = 5
    t = df.iloc[skip_log_rows:].reset_index(drop=True)
    t['start'] = (t['time'] - t.iloc[0]['time']).dt.total_seconds() # seconds from start
    t['start'] = t['start'] * speed_correction # recorder speed deviation compensation
    t['start'] = t['start'] + shift_sec
    t['end'] = t.shift(-1)['start']   
    mask = t['end'] - t['start'] > max_length_sec
    t.loc[mask, 'end'] = t.loc[mask, 'start'] + max_length_sec    
    t.dropna(subset=['end'], inplace=True) # drop last row
        
    if split_sec == 0:
        export_subtitles_file(t, out_file, formatter)
    else:
        n = 0
        while t.shape[0] > 0:
            export_subtitles_file(t[t.start < split_sec], out_file + str(n), formatter)
            t = t[t.start >= split_sec]
            t['start'] = t['start'] - split_sec
            t['end'] = t['end'] - split_sec
            n += 1

In [ ]:
# # speed_correction_fat_shark = 0.9972
# df = import_log_ardupilot('Bixler_3-2020-10-11.csv')
# export_subtitles(df, 'c:/temp/Video/2020-10-11_Bixler3/RC_000', skip_log_rows=645, split_sec=505)

In [ ]:
df = import_log_betaflight('c:/Dropbox/Projects/FPV/_Logs/Taranis/Titan-2021-05-12.csv')
export_subtitles(df, 'c:/Projects/FPV/_Video/2021-05-12_Titan/RC_000', skip_log_rows=0, shift_sec=24, split_sec=480)

In [ ]:
df = import_log_betaflight('c:/Dropbox/Projects/FPV/_Logs/Taranis/Titan-2021-05-15.csv')
export_subtitles(df, 'c:/Projects/FPV/_Video/2021-05-15_Titan/RC_000', skip_log_rows=0, shift_sec=55, split_sec=480)